In [3]:
from video_processor import VideoProcessor, merge_video_audio
from audiocraft.models import VidMuse
import scipy

ModuleNotFoundError: No module named 'video_processor'

In [ ]:
# Path to the video
video_path = 'sample.mp4'
# Initialize the video processor
processor = VideoProcessor()
# Process the video to obtain tensors and duration
local_video_tensor, global_video_tensor, duration = processor.process(video_path)

progress = True
USE_DIFFUSION = False

# Load the pre-trained VidMuse model
MODEL = VidMuse.get_pretrained('HKUSTAudio/VidMuse')
# Set generation parameters for the model based on video duration
MODEL.set_generation_params(duration=duration)

try:
    # Generate outputs using the model
    outputs = MODEL.generate([local_video_tensor, global_video_tensor], progress=progress, return_tokens=USE_DIFFUSION)
except RuntimeError as e:
    print(e)

# Detach outputs from the computation graph and convert to CPU float tensor
outputs = outputs.detach().cpu().float()


sampling_rate = 32000
output_wav_path = "vidmuse_sample.wav"
# Write the output audio data to a WAV file
scipy.io.wavfile.write(output_wav_path, rate=sampling_rate, data=outputs[0, 0].numpy())

output_video_path = "vidmuse_sample.mp4"
# Merge the original video with the generated music
merge_video_audio(video_path, output_wav_path, output_video_path)

### Missing Files